# Creating ARIMAPlus forcasting model

## load_test_data_tables in tests/system/conftest.py

In [3]:
import google.cloud.bigquery as bigquery

DATA_DIR="/usr/local/google/home/chelsealin/src/bigframes/tests/data/"
schema_filename=DATA_DIR + "time_series_schema.json"
data_filename=DATA_DIR + "time_series.jsonl"

time_series_table_id='bigframes-dev.chelsealin.time_series_0'

client = bigquery.Client(project='bigframes-dev')

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.schema = tuple(
    client.schema_from_json(schema_filename)
)
print(job_config.schema)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

with open(data_filename, "rb") as input_file:
    print(data_filename)
    job = client.load_table_from_file(
        input_file,
        time_series_table_id,
        job_config=job_config,
    )
job.result()

[SchemaField('parsed_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None), SchemaField('total_visits', 'INTEGER', 'NULLABLE', None, None, (), None)]
/usr/local/google/home/chelsealin/src/bigframes/tests/data/time_series.jsonl


LoadJob<project=bigframes-dev, location=US, id=93ef6ec3-26d5-42e3-9949-5ed61d8c56bc>

## time_series_arima_plus_model in tests/system/conftest.py: 

In [6]:
import bigframes.pandas as bpd

time_series_arima_plus_model_name = "bigframes-dev.chelsealin.time_series_arima_plus_0"
sql = f"""
CREATE OR REPLACE MODEL `{time_series_arima_plus_model_name}`
OPTIONS (
    model_type='ARIMA_PLUS',
    time_series_timestamp_col = 'parsed_date',
    time_series_data_col = 'total_visits'
) AS SELECT
    *
FROM `{time_series_table_id}`"""

client.query(sql).result()

In [7]:
time_series_arima_plus_model = bpd.read_gbq_model(time_series_arima_plus_model_name)

## test_arima_plus_predict_attribution_default

In [8]:
time_series_arima_plus_model.predict()

,forecast_timestamp,forecast_value,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,confidence_interval_lower_bound,confidence_interval_upper_bound
0,2017-08-02 00:00:00+00:00,2727.693349,190.614778,0.95,2354.763525,3100.623174,2354.763525,3100.623174
1,2017-08-03 00:00:00+00:00,2595.290749,222.093338,0.95,2160.774441,3029.807056,2160.774441,3029.807056
2,2017-08-04 00:00:00+00:00,2370.86767,255.645942,0.95,1870.707097,2871.028242,1870.707097,2871.028242


In [10]:
time_series_arima_plus_model.predict_attribution()

,time_series_timestamp,time_series_type,time_series_data,time_series_adjusted_data,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,trend,seasonal_period_yearly,seasonal_period_quarterly,seasonal_period_monthly,seasonal_period_weekly,seasonal_period_daily,holiday_effect,spikes_and_dips,step_changes,residual
0,2016-08-01 00:00:00+00:00,history,1711.0,505.716474,190.614736,<NA>,<NA>,<NA>,0.0,<NA>,<NA>,<NA>,169.611938,<NA>,<NA>,<NA>,1205.283526,336.104536
1,2016-08-02 00:00:00+00:00,history,2140.0,625.750155,190.614736,<NA>,<NA>,<NA>,338.716882,<NA>,<NA>,<NA>,287.033273,<NA>,<NA>,<NA>,1205.283526,308.966319
2,2016-08-03 00:00:00+00:00,history,2890.0,995.111101,190.614736,<NA>,<NA>,<NA>,549.970223,<NA>,<NA>,<NA>,445.140878,<NA>,<NA>,<NA>,1205.283526,689.605373
3,2016-08-04 00:00:00+00:00,history,3161.0,1408.363927,190.614736,<NA>,<NA>,<NA>,1005.271573,<NA>,<NA>,<NA>,403.092354,<NA>,<NA>,<NA>,1205.283526,547.352547
4,2016-08-05 00:00:00+00:00,history,2702.0,1381.96532,190.614736,<NA>,<NA>,<NA>,1236.276965,<NA>,<NA>,<NA>,145.688355,<NA>,<NA>,<NA>,1205.283526,114.751155
5,2016-08-06 00:00:00+00:00,history,1663.0,349.426733,190.614736,<NA>,<NA>,<NA>,1100.924343,<NA>,<NA>,<NA>,-751.49761,<NA>,<NA>,<NA>,1205.283526,108.289741
6,2016-08-07 00:00:00+00:00,history,1622.0,432.654477,190.614736,<NA>,<NA>,<NA>,1134.891238,<NA>,<NA>,<NA>,-702.236761,<NA>,<NA>,<NA>,1205.283526,-15.938002
7,2016-08-08 00:00:00+00:00,history,2815.0,1414.879653,190.614736,<NA>,<NA>,<NA>,1239.098627,<NA>,<NA>,<NA>,175.781026,<NA>,<NA>,<NA>,1205.283526,194.836821
8,2016-08-09 00:00:00+00:00,history,2851.0,1622.001182,190.614736,<NA>,<NA>,<NA>,1328.328933,<NA>,<NA>,<NA>,293.672249,<NA>,<NA>,<NA>,1205.283526,23.715292
9,2016-08-10 00:00:00+00:00,history,2757.0,1718.174551,190.614736,<NA>,<NA>,<NA>,1273.916936,<NA>,<NA>,<NA>,444.257615,<NA>,<NA>,<NA>,1205.283526,-166.458077
